Thursday To-Do List:
1. Get more pictures for testing
2. Figure out the proper evaluation metrics (signal/noise, etc...)
3. Make a seperate method for evaluation
4. Do statistical tests to prove SPIR is accurate
5. Make a method/function for testing and graphing results and saving images
6. Make a default list of parameters, make the parameters option optional, make it easier to change
7. Show that/if replacing unused atoms gives better results
8. Run tests on other toggles in the algorithm

In [1]:
import numpy as np
import random
import cv2
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time

%run Classes.py

In [2]:
paths = ['Compressed Images/cheese_board.jpg', 'Compressed Images/chicken_n_beans.jpg', 'Compressed Images/persimmon_tomato_salad.jpg', 'Compressed Images/octopus.jpg']
new_path = 'Reconstructed Images/tea_eggs'
path_orig = 'Compressed Images/tea_eggs.jpg'
path_corrupted = 'Compressed Images/tea_eggs_corrupted.jpg'

sam = Sampler(paths = paths, patch_shape = np.array([8,8]), num_samples = 500)
sam.add_filter('noise', std = 100)
learner = DictionaryLearner(L=20, K=200, sampler=sam, algo = 'OMP')
learner.update_step(use_orig=True)

img_orig = load_image(path_orig)
img_corrupt = sam.filter(img_orig)

cv2.imwrite(path_corrupted, img_corrupt)

cv2.imshow("Original Image", img_orig)
cv2.imshow("Corrupted Image", img_corrupt)
cv2.moveWindow("Corrupted Image", img_orig.shape[1], 0)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
D = learner.sparse_dictionary_learning(iters=20, output=True)
print("Dictionary Complete")

  0%|          | 0/20 [00:00<?, ?it/s]

Dictionary Complete


In [4]:
(recon, error) = learner.SPIR(path=path_corrupted, percent=.2, min_count=1)
print(f'Reconstruction Complete')
print(f'Error Est. = {error}')

cv2.imshow("Original Image", img_orig)
cv2.imshow("Corrupted Image", img_corrupt)
cv2.moveWindow("Corrupted Image", img_orig.shape[1], 0)

cv2.imshow('Reconstructed Image', recon)
cv2.waitKey(0)
cv2.destroyAllWindows()




  0%|          | 0/5519 [00:00<?, ?it/s]

C:\Users\raino\GitHub\Sparse-Dictionary-Learning\Classes.py:554: RuntimeWarning: invalid value encountered in floor_divide
  recon_img = recon_img // count


Reconstruction Complete
Error Est. = 39.81040120829459


In [38]:
print(f'Iters = 0, Error = {error}')
#print(f'Iters = 1, Error = 6.0670')
#print(f'Iters = 2, Error = 5.3670')
#print(f'Iters = 10, Error = {error}')

Iters = 0, Error = 40.237869559798675


In [6]:
cv2.imshow('temp image', recon)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def new_test_process(test_name, N=300, K=100, I=5, L=4, P=8, save_dic = False, generate_graph = False, variabel = None):
    print(f"Test: {test_name}")
    print(f"Params: N = {N}, K = {K}, I = {I}, L = {L}, P = {P}")

    patch_shape = np.array([P, P])

    sam = Sampler(paths=paths, patch_shape=patch_shape, num_samples=N)
    learner = DictionaryLearner(L=L, K=K, sampler=sam, algo='OMP')

    # Start the timer
    start_time = time.time()

    D = learner.sparse_dictionary_learning(iters=I, output=True)

    (recon, error) = learner.SPIR(path=path, percent=percent, min_count=min_count)

    # Calculate the run time
    run_time = time.time() - start_time

    # Save the reconstruction
    correct_path = 'Reconstructed Images/' + test_name + 'N' + str(N) + 'K' + str(K) + 'I' + str(I) + 'L' + str(L) + 'P' + str(P) + '.jpg'
    cv2.imwrite(correct_path, recon)

    # Print the results
    print(f"Error = {error}, Run Time = {run_time}")

    # Save the results
    #data[(N, K, I, L, P)] = [error, run_time]

    with open('Graphs/' + test_name + '_data.pkl', 'rb') as f:
        data = pickle.load(f)

    data[(N, K, I, L, P)] = [error, run_time]

    with open('Graphs/' + test_name + '_data.pkl', 'wb') as f:
        pickle.dump(data, f)

    if save_dic:

        with open('Graphs/' + test_name + '_dictionaries.pkl', 'rb') as f:
            dictionaries = pickle.load(f)

        dictionaries[(N, K, I, L, P)] = D

        with open('Graphs/' + test_name + '_dictionaries.pkl', 'wb') as f:
            pickle.dump(data, f)

    if generate_graph:
        xs =[x[variabel] for x in list(data.keys())]
        es = [y[0] for y in list(data.values())]
        ts = [y[1] for y in list(data.values())]


        # create the figure and first axis
        fig, ax1 = plt.subplots(figsize=(8, 6))

        # create the first line and first y-axis
        ax1.plot(xs, es, linestyle='-.', marker='s', color='blue', label='Error Est.')

        X_label = ['N', 'K', 'I', 'L', 'P']
        X_label = X_label[variabel]
        ax1.set_xlabel(X_label + ' Value')

        ax1.set_ylabel('Error Est.')
        ax2 = ax1.twinx()

        ax2.plot(xs, ts, linestyle='--', marker='^', color='green', label = 'Run Time')

        # rotate and adjust second y-axis label
        ax2.yaxis.set_label_coords(1.1, 0.5)
        ax2.set_ylabel('Run Time', rotation=-90, labelpad=10)

        ax1.set_title('Run Time and Error Est. with varying ' + X_label + ' values')

        # combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2)

        # save the graph as a PNG file
        plt.savefig('Graphs/' + test_name + '.png')

        plt.show()

